In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from dataclasses import dataclass

@dataclass
class MetOceanParameters:
    Hm0: float        # Significant wave height [m]
    Tp: float         # Peak wave period [s]
    T02: float        # Mean zero-crossing period [s]
    CS_total: float   # Depth-averaged current speed [m/s]
    WS160: float      # Wind speed at 160 m [m/s]
    WS160_adj: float  # Wind speed at 160 m (+7%) [m/s]
    WL_tot: float     # Total water level [m MSL]
    Hmax: float       # Maximum wave height [m]
    T_Hmax: float     # Wave period associated with Hmax [s]
    Cmax_SWL: float   # Max crest level w.r.t. SWL [m]
    Cmax_MSL: float   # Max crest level w.r.t. MSL [m]
    WL_total: float      # Total water level (HWL_tot or LWL_tot) [mMSL]
    WL_residual: float   # Residual water level (HWL_res or LWL_res) [m]

# -------------------------
# 200-year return period
# -------------------------

HWL = MetOceanParameters(
    Hm0=9.9,
    Tp=15.3,
    T02=9.8,
    CS_total=0.5,
    WS160=29.7,
    WS160_adj=31.8,
    WL_tot=1.3,
    Hmax=19.2,
    T_Hmax=12.1,
    Cmax_SWL=12.8,
    Cmax_MSL=14.6,
    WL_total=2.4,
    WL_residual=2.2
)

LWL = MetOceanParameters(
    Hm0=8.7,
    Tp=13.4,
    T02=8.5,
    CS_total=0.5,
    WS160=32.9,
    WS160_adj=35.2,
    WL_tot=-0.2,
    Hmax=16.6,
    T_Hmax=10.5,
    Cmax_SWL=11.0,
    Cmax_MSL=10.7,
    WL_total=-1.7,
    WL_residual=-1.4
)


## Parameters

In [ ]:
# General constants
g = 9.81 # m/s^2
rho = 1025 # kg/m^3
nu = 1.33e-6 # m^2/s

Kv = 1.1 # Velocity coefficient

## Armour Layer

In [11]:
def specific_density(rho_s=2650.0, rho_w=1000.0):
    """
    Calculates the specific density Δ_s.

    Parameters
    ----------
    rho_s : float
        Density of rock material [kg/m³]
    rho_w : float, optional
        Density of water [kg/m³] (default = 1000)

    Returns
    -------
    Delta_s : float
        Specific density [-]
    """
    return (rho_s - rho_w) / rho_w

def dimensionless_particle_diameter(d50, Delta_s, nu, g):
    """
    Calculates the dimensionless particle diameter d_*.

    Parameters
    ----------
    d50 : float
        Median particle diameter [m]
    Delta_s : float
        Specific density [-]
    nu : float
        Kinematic viscosity of water [m²/s]
    g : float, optional
        Gravitational acceleration [m/s²] (default = 9.81)

    Returns
    -------
    d_star : float
        Dimensionless particle diameter [-]
    """
    return d50 * ((Delta_s * g) / (nu**2)) ** (1.0 / 3.0)   


def bereken_armor_diameter(u_c, h, psi_c=0.03, K_s=1.0, K_v=1.0, delta=1.59):
    """
    Berekent de karakteristieke steendiameter d_n50 van een armorlaag (shields).

    Parameters
    ----------
    u_c : float
        Kritische, diepte-gemiddelde stroomsnelheid bij uniforme stroming [m/s]
    h: float
        Waterdiepte [m]
    psi_c : float, optional
        Drempel-van-beweging parameter (standaard 0.03)
    K_s : float, optional
        Sterkte-reductiefactor voor stenen op een talud [-]
    K_v : float, optional
        Snelheids-/turbulentiefactor [-]
    delta : float, optional
        Relatieve dichtheid (Δ = (ρ_s − ρ_w) / ρ_w) [-]

    Returns
    -------
    d : float
        Benodigde steendiameter [m]
    """
    C = 50.0 # Standaard Chezy-coëfficiënt
    
    for _ in range(10):
        d_n50 = (u_c**2) / (K_s * psi_c * delta * C**2)
        C = 18*np.log10(12*h/2*d_n50)
    d_n50 = (K_v ** 2 * u_c**2) / (K_s * psi_c * delta * C**2)

    return d_n50



In [ ]:
calculate_armor_diameter =  (u_c, C, psi_c=0.03, K_s=1.0, K_v=1.0, delta_C=1.0) => 